In [5]:
badscript = '''contract get_fib_n(n:int){
    let last_fib_calculated : int = 0;

    entry get_fib(n: int){
        let result: int = fib(n);
        last_fib_calculated = result;
        let a : int = b;
    }

    func fib(n: int) : int{
        let x : int = last_fib_calculated;
        if (n <= 1) {
            return n;
        }
        else {
            
            let a: int = n - 1;
            let b: int = n - 2;
            return fib(a) + fib(b);
        }
    }

    func fib(n: int) : int{
        let x : int = last_fib_calculated;
    }
}'''

In [10]:
script = '''
contract get_fib_n(n:int){
    let last_fib_calculated: int = 0;

    func fib(n: int) : int{
        if (n <= 1) {
            return n;
        }
        else {
            let a: int = n - 1;
            let b: int = n - 2;
            return fib(a) + fib(b);
        }
    }

    entry get_fib_n(n: int){
        let result: int = fib(n);
        last_fib_calculated = result;
    }

    
}
'''

In [11]:
from lexer.sly_lexer import TzScriptLexer, process_lexer_tokens 
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry,lessthanequal,greaterthanequal, iniquelaty, lessthan,greaterthan,equalequal, returnx
from parser.slr_parser import SLR1Parser, build_slr_ast

lexer = TzScriptLexer()
lexer_tokens = list(lexer.tokenize(script))
tokens = process_lexer_tokens(lexer_tokens)

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)

0 	 <program> -> .contractid(<param-list>){<stat_list>}, 
	 S' -> .<program>,  

1 	 <program> -> contract.id(<param-list>){<stat_list>},  

2 	 <program> -> contractid.(<param-list>){<stat_list>},  

3 	 <param-list> -> .<param>, 
	 <program> -> contractid(.<param-list>){<stat_list>}, 
	 <param-list> -> .<param>,<param-list>, 
	 <param> -> .id:type,  

4 	 <param> -> id.:type,  

5 	 <param> -> id:.type,  

6 	 <param> -> id:type.,  

7 	 <program> -> contractid(<param-list>.){<stat_list>},  

8 	 <program> -> contractid(<param-list>).{<stat_list>},  

9 	 <stat> -> .<if-stat>, 
	 <stat_list> -> .<stat>, 
	 <program> -> contractid(<param-list>){.<stat_list>}, 
	 <stat> -> .<var-call>, 
	 <stat> -> .<return-stat>, 
	 <if-stat> -> .if(<expr>){<stat_list>}, 
	 <let-var>> -> .letid:type=<expr>;, 
	 <def-func> -> .funcid(<param-list>):type{<stat_list>}, 
	 <def-entry> -> .entryid(<param-list>){<stat_list>}, 
	 <stat> -> .<let-var>>, 
	 <else-stat> -> .else{<stat_list>}, 
	 <return-stat> ->

In [12]:
productions, operations = derivation


from visitors.string_rep_visitor import FormatVisitor
ast = build_slr_ast(productions, operations, tokens)
format = FormatVisitor()
print(format.visit(ast))

\__ProgramNode: contract node.idx(n : int) [<stat>; ... <stat>;]
	\__AttrDeclarationNode: n : int
		\__VarDeclarationNode: let last_fib_calculated = <expr> : int
			\__ ConstantNumNode: 0
		\__FuncDeclarationNode: def fib(n : int) : :
			\__AttrDeclarationNode: n : int
				\__IfNode: if <expr> then [<stat>; ... <stat>;]
					\__<expr> LessThanEqualNode <expr>
						\__VariableNode: n
						\__ ConstantNumNode: 1
					\__ReturnStatementNode: return <expr>
						\__VariableNode: n
				\__ElseNode: else [<stat>; ... <stat>;]
					\__VarDeclarationNode: let a = <expr> : int
						\__<expr> MinusNode <expr>
							\__VariableNode: n
							\__ ConstantNumNode: 1
					\__VarDeclarationNode: let b = <expr> : int
						\__<expr> MinusNode <expr>
							\__VariableNode: n
							\__ ConstantNumNode: 2
					\__ReturnStatementNode: return <expr>
						\__<expr> PlusNode <expr>
							\__CallNode: fib(<expr>, ..., <expr>)
								\__VariableNode: a
							\__CallNode: fib(<expr>, ..., <expr>)
				

In [13]:
from visitors.semantic_check_visitor import SemanticCheckerVisitor
semantic_checker = SemanticCheckerVisitor()
semantic_checker.visit(ast)


fib
fib
n arg id
fib
estoy en last_fib_calculated [<visitors.scope.VariableInfo object at 0x107b41cd0>, <visitors.scope.VariableInfo object at 0x107b57510>, <visitors.scope.VariableInfo object at 0x107b552d0>]


[]